In [ ]:
#loading libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import fiona
import pyproj
import contextily
import plotly.express as px
import dash_mantine_components as dmc


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
path = os.getcwd()

### Loading demographic merge from drive

In [2]:
# Steps to upload Geojson google drive file:
# Share -> "Copy Link"
# Download
# Right click download anyway -> Inspect
# Copy the link within Form-ID within <doctype>

demographic = gpd.read_file('https://drive.google.com/u/0/uc?id=1bMnHgBkO2oXWKzGieNZDlAj0lLEil9ae&export=download&confirm=t&uuid=7d19032f-02e8-43a0-953b-1abfabfeb088&at=ALt4Tm0IkQtPWwgV-fCImYX1LZVC:1689710380703')

**Code for the autoplot function**

In [6]:
def demographic_plot(frame, column, counties = ['Monterey','Los Angeles','Kern','Fresno','Mendocino'], geometry_level = 'county'): 
    if (geometry_level in list(frame.geometry_type.unique())):
            frame = frame.loc[frame['geometry_type'] == geometry_level]
    else: return 'invalid geometry level'
    
    if (counties != []):
        frame = frame.loc[frame['county_name'].isin(counties)]
        
    if column in list(frame.columns.values):
        #frame = frame[[column,'geometry']]
        
        frame.reset_index(inplace = True)
        frame.set_index('index',inplace = True)
        
        fig = px.choropleth_mapbox(frame,
                                   geojson=frame.geometry,
                                   locations=frame.index,
                                   color = column,
                                   mapbox_style="carto-positron",
                                   center = {"lat":37.1661,"lon":-119.4494},
                                   zoom=5,
                                   color_continuous_scale="Viridis"
                                   )
        
        fig.update_geos(fitbounds="locations", visible=False)
        
        return fig, frame
    else: 
        return "invalid column name"

**Experimental Code**

In [8]:
# making a dictionay for pesticide names
di2 = dict(zip(['','carcinogens','endo_disrupt','ch_I','fumigants','neonicotinoids','repro_dev'],['','carcinogens','endo_disrupt','ch_I','fumigants','neonicotinoids','repro_dev']))

In [6]:
# making a dictionay for counties names
di = dict(zip(list(demographic.county_name.unique()), list(demographic.county_name.unique())))

In [21]:
# Running dash app...
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

app.layout = html.Div(
    [
        html.Div(
            [
                html.P("Select plotting unit:"),
                dcc.RadioItems(
                    id='geometry_select', 
                    options=[{'label':'Tract','value':'tract'},{'label':'County','value':'county'}],
                    value="tract",
                    inline=True
                ),         
            ],
            style=dict(width='15%')
        ),
        html.Div(
            [
                html.H4('Pesticide Use Plotter'),
                html.P("Select statistic:"),
                dcc.RadioItems(
                    id='column', 
                    options=[{"label": "Percent white alone", "value": "pct_nh_white_alone"},
                            {"label": "Percent black or african american alone", "value": "pct_nh_black_or_african_american_alone"},
                            {"label":"Percent Amercan Indian and Alaske Native", "value": "pct_nh_american_indian_and_alaska_native_alone"},
                            {"label": "Percent Asian alone", "value": "pct_nh_asian_alone"},
                            {"label": "Percent Hispanic or Latino alone", "value": "pct_hispanic_or_latino"}
                    ],
                    value="pct_nh_white_alone"
                ),
                dcc.Graph(id="graph"),
                dcc.Dropdown(options = [{'label': v, 'value': k} for k, v in {'Santa Clara': 'Santa Clara', 'Orange': 'Orange', 'Alameda': 'Alameda',
                                                                              'Kern': 'Kern', 'San Bernardino': 'San Bernardino', 'Napa': 'Napa',
                                                                              'El Dorado': 'El Dorado', 'San Francisco': 'San Francisco', 'Monterey': 'Monterey',
                                                                              'Contra Costa': 'Contra Costa', 'Los Angeles': 'Los Angeles', 'San Joaquin': 'San Joaquin',
                                                                              'Sacramento': 'Sacramento', 'Placer': 'Placer', 'Sonoma': 'Sonoma',
                                                                              'Riverside': 'Riverside', 'Sutter': 'Sutter', 'Del Norte': 'Del Norte',
                                                                              'Ventura': 'Ventura', 'Tulare': 'Tulare', 'Santa Barbara': 'Santa Barbara',
                                                                              'Yolo': 'Yolo', 'Alpine': 'Alpine', 'Stanislaus': 'Stanislaus',
                                                                              'Trinity': 'Trinity', 'San Mateo': 'San Mateo', 'Solano': 'Solano',
                                                                              'Amador': 'Amador', 'Fresno': 'Fresno', 'Tehama': 'Tehama',
                                                                              'San Luis Obispo': 'San Luis Obispo', 'Merced': 'Merced', 'Butte': 'Butte',
                                                                              'Lake': 'Lake', 'San Diego': 'San Diego', 'Madera': 'Madera',
                                                                              'Lassen': 'Lassen', 'Imperial': 'Imperial', 'Mariposa': 'Mariposa',
                                                                              'Nevada': 'Nevada', 'Yuba': 'Yuba', 'Tuolumne': 'Tuolumne',
                                                                              'Marin': 'Marin', 'Calaveras': 'Calaveras', 'Santa Cruz': 'Santa Cruz',
                                                                              'Shasta': 'Shasta', 'Humboldt': 'Humboldt', 'Siskiyou': 'Siskiyou',
                                                                              'Mendocino': 'Mendocino', 'San Benito': 'San Benito', 'Plumas': 'Plumas',
                                                                              'Sierra': 'Sierra', 'Kings': 'Kings', 'Colusa': 'Colusa',
                                                                              'Modoc': 'Modoc', 'Inyo': 'Inyo','Glenn': 'Glenn', 'Mono': 'Mono'}.items()],
                    id = 'county_multi_select',
                    value =  ['Monterey','Los Angeles','Kern','Fresno','Mendocino'],
                    multi=True
                )
            ],
            style=dict(width='85%'))
    ],
    style=dict(display='flex')
)
@app.callback(
    Output("graph", "figure"),
    Input("column", "value"),
    Input('county_multi_select', 'value'),
    Input("geometry_select", "value")
    )#,

def update_graph(column, county_multi_select, geometry_select):#County, yearslider):
    return demographic_plot(demographic, column, counties = county_multi_select,geometry_level = geometry_select)#,county = [County], year = yearslider)

def demographic_plot(frame, column, counties = ['Monterey','Los Angeles','Kern','Fresno','Mendocino'], geometry_level = 'county'): 
    if (geometry_level in list(frame.geometry_type.unique())):
            frame2 = frame.loc[frame['geometry_type'] == geometry_level]
    else: return 'invalid geometry level'
    
    if (counties != []):
        frame2 = frame.loc[frame['county_name'].isin(counties)]
        
    if column in list(frame2.columns.values):
        #frame = frame[[column,'geometry']]
        
        frame2.reset_index(inplace = True)
        frame2.set_index('index',inplace = True)
        
        fig = px.choropleth_mapbox(frame2,
                                   geojson=frame2.geometry,
                                   locations=frame2.index,
                                   color = column,
                                   mapbox_style="carto-positron",
                                   center = {"lat":37.1661,"lon":-119.4494},
                                   zoom=5,
                                   color_continuous_scale="Viridis"
                                   )
        
        fig.update_geos(fitbounds="locations", visible=False)
        
        return fig
    
    else: 
        return "invalid column name"


if __name__ == "__main__":
    app.run_server(debug=True)